<a href="https://colab.research.google.com/github/csabi0312/DeepLProject/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Packages

In [58]:
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split


# Setting the random seed
seed_value = 42
random.seed(seed_value)

#Data manipulations

In [59]:
#Loading the questions
data = pd.read_csv("https://raw.githubusercontent.com/csabi0312/DeepLProject/main/train.csv",index_col=0)

In [60]:
data.head()

,prompt,A,B,C,D,E,answer
id,,,,,,,
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,D
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,A
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,A
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,C
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,D


In [61]:
# Creating a dictionary to map the values to numbers
mapping = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

# Replacing the values in the 'answer' column
data['answer'] = data['answer'].replace(mapping)
data.head()

,prompt,A,B,C,D,E,answer
id,,,,,,,
0,Which of the following statements accurately d...,MOND is a theory that reduces the observed mis...,MOND is a theory that increases the discrepanc...,MOND is a theory that explains the missing bar...,MOND is a theory that reduces the discrepancy ...,MOND is a theory that eliminates the observed ...,3
1,Which of the following is an accurate definiti...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,Dynamic scaling refers to the non-evolution of...,Dynamic scaling refers to the evolution of sel...,0
2,Which of the following statements accurately d...,The triskeles symbol was reconstructed as a fe...,The triskeles symbol is a representation of th...,The triskeles symbol is a representation of a ...,The triskeles symbol represents three interloc...,The triskeles symbol is a representation of th...,0
3,What is the significance of regularization in ...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,Regularizing the mass-energy of an electron wi...,2
4,Which of the following statements accurately d...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,The angular spacing of features in the diffrac...,3


In [62]:
#Train-Val-Test split:
# Splitting the DataFrame into training, validation, and test datasets with a 2:1:1 ratio
train, temp = train_test_split(data, test_size=0.5, random_state=42)
val, test = train_test_split(temp, test_size=0.5, random_state=42)

print(len(train))
print(len(val))
print(len(test))

100
50
50


We want to merge the questions with the possible answers and create a dataset with two columns: merged text and answer as an integer label.<br> <br>
To reduce overfitting and enrich the data we will consider two possible random permuations of the answers and merge them with the question.<br>
This means that for each question there will be 2 datapoints in the new dataset, where they are merged with their answers 2 times in a random order.

In [63]:
def shuffle_list_with_index(list_, original_index):
    # Shuffling the list without changing the original list
    shuffled_list = random.sample(list_, len(list_))

    # Finding the new index of the original right element
    right_word = list_[original_index]
    new_index = shuffled_list.index(right_word)

    return shuffled_list, new_index

In [64]:
#Testing the shuffling and the merging for the first row of the dataset
row = train.iloc[0]
question = row[0]
answers = [row[i] for i in range(1,6)]
label = row[6]

new_answers, new_label = shuffle_list_with_index(answers,label)

result1 = ['\n'.join([question] + answers),label]
result2 = ['\n'.join([question] + new_answers), new_label]

for i in result1:
  print(i)
print('\n')
for j in result2:
  print(j)

Which of the following statements accurately describes the relationship between the dimensions of a diffracting object and the angular spacing of features in the diffraction pattern?
The angular spacing of features in the diffraction pattern is indirectly proportional to the dimensions of the object causing the diffraction. Therefore, if the diffracting object is smaller, the resulting diffraction pattern will be narrower.
The angular spacing of features in the diffraction pattern is directly proportional to the dimensions of the object causing the diffraction. Therefore, if the diffracting object is smaller, the resulting diffraction pattern will be narrower.
The angular spacing of features in the diffraction pattern is independent of the dimensions of the object causing the diffraction. Therefore, if the diffracting object is smaller, the resulting diffraction pattern will be the same as if it were big.
The angular spacing of features in the diffraction pattern is inversely proportio

In [65]:
# Initializing an empty list
X_t=[]

# Looping through the 'train' DataFrame using tqdm for progress visualization
for index in tqdm(range(len(train))):
    # Extracting the current row from the DataFrame
    row = train.iloc[index]
    # Extracting the question from the current row
    question = row[0]
    # Extracting the answers from the current row
    answers = [row[i] for i in range(1,6)]
    # Extracting the label from the current row
    label = row[6]

    # Shuffling the answers and updating the label using the 'shuffle_list_with_index' function
    new_answers, new_label = shuffle_list_with_index(answers,label)

    # Combining the question and answers with a new line and adding the original label to the list
    result1 = ['\n '.join([question] + answers),label]
    # Combining the question and shuffled answers with a new line and adding the updated label to the list
    result2 = ['\n '.join([question] + new_answers), new_label]

    # Appending both results to the list 'X_t'
    X_t.append(result1)
    X_t.append(result2)

# Creating a new DataFrame 'df_train' from the list 'X_t' with the specified column names
df_train = pd.DataFrame(X_t, columns=["text","label"])

# Displaying the first few rows of the newly created DataFrame
df_train.head()

100%|██████████| 100/100 [00:00<00:00, 8090.40it/s]


,text,label
0,Which of the following statements accurately d...,3
1,Which of the following statements accurately d...,4
2,What is the second law of thermodynamics?\n Th...,4
3,What is the second law of thermodynamics?\n Th...,0
4,What is radiometric dating?\n Radiometric dati...,1


In [66]:
X_v=[]

for index in tqdm(range(len(val))):
  row = val.iloc[index]
  question = row[0]
  answers = [row[i] for i in range(1,6)]
  label = row[6]

  result1 = ['\n '.join([question] + answers),label]
  X_v.append(result1)

df_val = pd.DataFrame(X_v, columns=["text","label"])
df_val.shape

100%|██████████| 50/50 [00:00<00:00, 5734.31it/s]


(50, 2)

In [67]:
X_e=[]

for index in tqdm(range(len(test))):
  row = test.iloc[index]
  question = row[0]
  answers = [row[i] for i in range(1,6)]
  label = row[6]

  result1 = ['\n '.join([question] + answers),label]
  X_e.append(result1)

df_test = pd.DataFrame(X_e, columns=["text","label"])
df_test.shape

100%|██████████| 50/50 [00:00<00:00, 9042.57it/s]


(50, 2)

Now we have the 3 neccesary datasets for training and testing:


*   df_train: enriched train dataset
*   df_val : validation dataset
*   df_test : test dataset

